In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
input_df = pd.read_csv('../input/train.csv')
submit_df = pd.read_csv('../input/test.csv')

In [ ]:
passengerID = submit_df.PassengerId

In [ ]:
df = pd.concat([input_df, submit_df],sort=False)

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df.drop('index',axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df['Title']=df.Name.apply(lambda x:x.split(",")[1].split(".")[0].split(" ")[1])

In [ ]:
grouped = df.groupby(['Sex','Pclass','Title'])

In [ ]:
grouped.Age.median()

In [ ]:
df['Title'].value_counts()

In [ ]:
df.head()

In [ ]:
df.Age = grouped.Age.apply(lambda x:x.fillna(x.median()))

In [ ]:
df.Age = df.Age.fillna(28)

In [ ]:
normalized_titles = {'Mr'   :'Mr' , 
                     'Miss'  :'Miss',
'Mrs': 'Mrs',
'Master' : 'Master',
    'Rev' : 'Dr',
 'Col' : 'Dr' , 
 'Major' : 'Dr', 
 'Mlle'  :'Miss',
 'Ms'  : 'Miss',
 'Don'  :  'Mr',
 'Sir'    : 'Dr',
 'Lady'     :'Dr',
 'Jonkheer' :'Mr', 
 'Capt'  : 'Dr' ,
 'the Countess' : 'Miss' ,
 'Dona'  :'Mrs',
 'Mme'    :'Miss' }

In [ ]:
df['Title']=df['Title'].map(normalized_titles)

In [ ]:
df['Title'].value_counts()

In [ ]:
df.Cabin = df.Cabin.fillna('U')

In [ ]:
df.Embarked=df.Embarked.fillna(df.Embarked.value_counts().index[0])

In [ ]:
df.Fare[df.Fare.isnull()] = df['Fare'].median()

In [ ]:
df.info()

In [ ]:
df['Family Size'] = df.Parch + df.SibSp + 1

In [ ]:
df.Cabin = df.Cabin.map(lambda x:x[0])

In [ ]:
df.head()

In [ ]:
df.Sex.apply(lambda x:len(x))

In [ ]:
df.Sex = df.Sex.map({'male':0,'female':1})

In [ ]:
#create dummies for categorical features
pclass = pd.get_dummies(df.Pclass,prefix='Pclass')
embark =pd.get_dummies(df.Embarked,prefix='Embarked')
title = pd.get_dummies(df.Title,prefix='Title')
cabin = pd.get_dummies(df.Cabin,prefix='Cabin')
#Concatenate all dummy columns with main dataset
df_dummies = pd.concat([df,pclass,title,cabin,embark], axis=1)

In [ ]:
df_dummies.drop(['Name','Ticket','Title','Embarked','Pclass','Cabin'],axis=1,inplace=True)

In [ ]:
df_dummies.head()

In [ ]:
test = df_dummies[df_dummies.Survived.isnull()]

In [ ]:
train = df_dummies[df_dummies.Survived.isnull()==False]

In [ ]:
train.drop('PassengerId',axis=1,inplace=True)
test.drop('PassengerId',axis=1,inplace=True)

In [ ]:
X_train = train.drop('Survived',axis=1)

In [ ]:
y_train =train['Survived']

In [ ]:
X_test = test.drop('Survived',axis=1)

In [ ]:
y_test =test['Survived']

In [ ]:
X_train.head()

In [ ]:
from sklearn.feature_selection import mutual_info_classif
features = mutual_info_classif(X_train, y_train)

In [ ]:
features = pd.DataFrame(features, index=X_train.columns)
features.sort_values(by=0,ascending=False)

In [ ]:
X_train = X_train.drop(['Cabin_A', 'Cabin_D', 'Cabin_E', 'Cabin_F','Title_Dr','Embarked_Q','Pclass_2'],axis=1)

In [ ]:
X_test = X_test.drop(['Cabin_A', 'Cabin_D', 'Cabin_E', 'Cabin_F','Title_Dr','Embarked_Q','Pclass_2'],axis=1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
#from sklearn.ensemble import RandomForestClassifier

In [ ]:
#  forest_paramss = dict(max_depth = [n for n in range(5,14)],
#                     min_samples_split = [n for n in range(4,11)],
#                     min_samples_leaf = [n for n in range(2,5)],
#                     n_estimators = [n for n in range(50,60,3)],
#                       max_features =[n for n in range(2,27)])

# forest_paramss = dict(max_depth = [n for n in range(1,20)],
#                     min_samples_split = [n for n in range(2,20)],
#                     min_samples_leaf = [n for n in range(1,20)],
#                     max_features = [n for n in range(1,20)])

In [ ]:
#rfc = RandomForestClassifier()
#dt = DecisionTreeClassifier()
# gnb = GaussianNB()
# lr = LogisticRegression()
# xgb = XGBClassifier()
# rfc = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
#             max_depth=12, max_features='auto', max_leaf_nodes=None,
#             min_impurity_decrease=0.0, min_impurity_split=None,
#             min_samples_leaf=2, min_samples_split=9,
#             min_weight_fraction_leaf=0.0, n_estimators=40, n_jobs=1,
#             oob_score=False, random_state=None, verbose=0,
#             warm_start=False)
rfc = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=12, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=4,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [ ]:
#forest_cv = GridSearchCV(estimator=rfc, param_grid=forest_paramss,cv=5)
#vc = VotingClassifier(estimators=[('rfc',rfc),('lr',lr),('xgb',xgb),('dt',dt),('gnb',gnb)],voting='hard')

In [ ]:
#forest_cv.fit(X_train, y_train)
rfc.fit(X_train, y_train)

In [ ]:
pred = rfc.predict(X_test)

In [ ]:
# print("Best score {}".format(forest_cv.best_score_))
# print("Best estimator {}".format(forest_cv.best_estimator_))

In [ ]:
#pred = forest_cv.predict(X_test)

In [ ]:
output = pd.DataFrame({'PassengerId':passengerID,'Survived':pred})

In [ ]:
output.head()

In [ ]:
output['Survived'] = output['Survived'].astype(int)

In [ ]:
output.to_csv('output.csv', index=False)